In [1]:
#| include: false

import os
import pandas as pd
import numpy as np
# import plotly.express as px
import toml
# import psrc_theme
from pathlib import Path
import summary_data

# to show plotly figures in quarto HTML file
# import plotly.io as pio
# pio.renderers.default = "plotly_mimetype+notebook_connected"
# pio.templates.default = "simple_white+psrc_color" # set plotly template

config = summary_data.CONFIG
valid_config = toml.load(Path(Path.cwd(), '..\\..\\..\\..\\configuration', 'validation_configuration.toml'))
all_runs = summary_data.ALL_RUNS

## transit boardings

### boardings by mode

In [2]:

# test = pd.read_csv(os.path.join('C:/workspace/sc_2050_base_year_2023/outputs/transit/transit_line_results.csv'))

In [3]:
# Load model results and calculate modeled daily boarding by line
df = summary_data.load_agg_data('transit/transit_line_results.csv')
df_transit_line = (
    df.groupby(['source', 'route_code', 'mode', 'agency_code'])
    .agg({"description": "first", "boardings": "sum"})
    .reset_index()
)
df_transit_line['agency'] = df_transit_line['agency_code'].astype('int').astype('str').map(config['agency_lookup'])

In [4]:
tab = df_transit_line.groupby(['source','mode'])[['boardings']].sum().\
        reset_index().\
        pivot(index='source', columns='mode')['boardings'].\
        rename_axis(columns={'mode': 'Transit boardings by mode'})
tab['Total'] = tab.sum(axis=1)

display(tab.style.format('{:,.0f}'))


Transit boardings by mode,b,c,f,p,r,Total
source,,,,,,
2023 base year,"374,898","6,790","8,940","4,345","97,697","492,670"
2050,"971,722","7,051","24,501","7,535","396,932","1,407,741"
2050 (no telecommute),"1,012,878","8,741","26,788","8,118","421,416","1,477,941"


### boardings by agency

In [5]:

tab = df_transit_line.groupby(['source','agency'])[['boardings']].sum().\
        reset_index().\
        pivot(index='source', columns='agency')['boardings'].\
        rename_axis(columns={'agency': 'Transit boardings by agency'})
tab['Total'] = tab.sum(axis=1)

display(tab.style.format('{:,.0f}'))

Transit boardings by agency,Community Transit,Everett Transit,King County Metro,Kitsap Transit,Pierce Transit,Sound Transit,Washington Ferries,Total
source,,,,,,,,
2023 base year,"27,193","4,548","282,075","10,875","16,165","145,482","6,334","492,670"
2050,"131,925","25,916","718,238","28,987","57,739","432,650","12,286","1,407,741"
2050 (no telecommute),"134,040","26,774","752,154","31,779","59,972","459,798","13,425","1,477,941"


In [6]:
df = df_transit_line[df_transit_line['mode'].isin(['b'])].copy()
tab = df.groupby(['source','agency'])[['boardings']].sum().\
        reset_index().\
        pivot(index='source', columns='agency')['boardings'].\
        rename_axis(columns={'agency': 'Transit boardings by agency'})
tab['Total'] = tab.sum(axis=1)

display(tab.style.format('{:,.0f}'))

Transit boardings by agency,Community Transit,Everett Transit,King County Metro,Kitsap Transit,Pierce Transit,Sound Transit,Total
source,,,,,,,
2023 base year,"27,193","4,548","279,030","5,072","16,165","42,890","374,898"
2050,"131,925","25,916","708,920","18,554","57,739","28,668","971,722"
2050 (no telecommute),"134,040","26,774","742,238","20,213","59,972","29,641","1,012,878"


In [7]:
### boardings by key routes

# def show_key_route_boardings(route_list):
#         df = df_transit_line[df_transit_line['route_code'].isin(route_list)].copy()
#         tab = df.groupby(['source','description'])[['boardings']].sum().\
#                 reset_index().\
#                 pivot(index='source', columns='description')['boardings'].\
#                 rename_axis(columns={'description': 'Transit boardings by key routes'})
#         tab['Total'] = tab.sum(axis=1)

#         display(tab.style.format('{:,.0f}', na_rep="-"))

In [8]:
# # Ferry
# show_key_route_boardings([4200,4201,4203,4204,5001,5002,5003,5004,5005,5006,5007,1973,1975])
# # Rail
# show_key_route_boardings([6996,6999,6998])
# # RapidRide, busiest buses, streetcar
# show_key_route_boardings([1671,1672,1673,1674,1675,1676,6550,1007,1040,1997,1998])

In [9]:
### boardings by light rail station
# don't have station names

# df = summary_data.load_agg_data('transit/light_rail_boardings.csv')
# df